This script takes the **Fisheye8K** dataset and creates new splits:

- Data from only a single camera is selected
- The rare class "Trucks" is chosen as an anomaly
- The train split does not include any "Trucks".
- The val split includes only frames with "Trucks"

With the new **fisheye8k_anomaly_detection** dataset we can learn a normality without "Trucks" and treat them as outliers during validation. This way we might be able to detect new images with "Trucks".

In [1]:
import fiftyone as fo
from fiftyone import ViewField as F
from PIL import Image, ImageDraw
import plotly.graph_objects as go
import os

import sys

sys.path.append("..")

from utils.selector import select_by_class

In [2]:
dataset = fo.load_dataset(
    "fisheye8k"
)  # Expects that the dataset was once loaded through the Data Engine
view_cam1 = dataset.match(F("location") == "cam1")

In [3]:
fo.launch_app(view_cam1)
# Analysis of the class distribution showed that "Truck" is a rare class
anomaly_class = "Truck"

In [4]:
view_train = select_by_class(
    view_cam1, classes_in=[], classes_out=[anomaly_class]
)  # Build training dataset (no anomaly_class)

view_val = select_by_class(
    view_cam1, classes_in=[anomaly_class], classes_out=[]
)  # Build validation dataset (1-n anomaly_class in each frame)

print(f"Train set has {len(view_train)} samples; val set has {len(view_val)} samples")

Train set has 351 samples; val set has 49 samples


In [5]:
# https://github.com/voxel51/fiftyone/issues/1952

export_root = "../output/datasets/"
dataset_name = "fisheye8k_anomaly_detection"
export_dir = os.path.join(export_root, dataset_name)
label_field = "ground_truth"

classes = dataset.distinct(
    "ground_truth.detections.label"
)  # Sorted list of all observed labels in a given field

dataset_splits = ["train", "val"]
dataset_type = fo.types.YOLOv5Dataset

view_train.export(
    export_dir=export_dir,
    dataset_type=dataset_type,
    label_field=label_field,
    split=dataset_splits[0],
    classes=classes,
)

view_val.export(
    export_dir=export_dir,
    dataset_type=dataset_type,
    label_field=label_field,
    split=dataset_splits[1],
    classes=classes,
)

Directory '../output/datasets/fisheye8k_anomaly_detection' already exists; export will be merged with existing files
 100% |█████████████████| 351/351 [5.8s elapsed, 0s remaining, 62.6 samples/s]      
Directory '../output/datasets/fisheye8k_anomaly_detection' already exists; export will be merged with existing files
 100% |███████████████████| 49/49 [944.2ms elapsed, 0s remaining, 51.9 samples/s]      


This section takes bounding boxes and generates masks in the mvtec-ad format for the validation.

mvtec-ad: Black png image with white pixels where ground truth is

In [ ]:
# Load the dataset we just generated
if dataset_name in fo.list_datasets():
    fo.delete_dataset(dataset_name)
    print("Existing dataset " + dataset_name + " was deleted.")

dataset = fo.Dataset(dataset_name)
for split in dataset_splits:
    dataset.add_dir(
        dataset_dir=export_dir,
        dataset_type=dataset_type,
        split=split,
        tags=split,
    )
dataset.compute_metadata()
anomalous_view = dataset.match_tags(
    "val", "test"
)  # If it got pre-processed, also select test samples
print(f"Processing {len(anomalous_view)} val samples")

export_folder = "fisheye8k_anomaly_detection_masks"
export_dir_masks = os.path.join(export_root, export_folder)
os.makedirs(export_dir_masks, exist_ok=True)

for sample in anomalous_view.iter_samples(progress=True):
    img_width = sample.metadata.width
    img_height = sample.metadata.height
    mask = Image.new("L", (img_width, img_height), 0)  # Create a black image
    draw = ImageDraw.Draw(mask)
    for bbox in sample.ground_truth.detections:
        if bbox.label == anomaly_class:
            # Convert V51 format to image format

            x_min_rel, y_min_rel, width_rel, height_rel = bbox.bounding_box
            x_min = int(x_min_rel * img_width)
            y_min = int(y_min_rel * img_height)
            x_max = int((x_min_rel + width_rel) * img_width)
            y_max = int((y_min_rel + height_rel) * img_height)

            # draw.rectangle([x0, y0, x1, y1], fill=255)  # [x0, y0, x1, y1]
            draw.rectangle([x_min, y_min, x_max, y_max], fill=255)  # [x0, y0, x1, y1]

    # Save the mask
    filename = os.path.basename(sample.filepath).replace(".jpg", ".png")
    mask.save(os.path.join(export_dir_masks, f"{filename}"))

Existing dataset fisheye8k_anomaly_detection was deleted.
 100% |█████████████████| 351/351 [5.1s elapsed, 0s remaining, 71.4 samples/s]      
 100% |███████████████████| 49/49 [710.2ms elapsed, 0s remaining, 69.2 samples/s]      
Computing metadata...
 100% |█████████████████| 400/400 [18.2s elapsed, 0s remaining, 26.1 samples/s]      
Processing 49 val samples
 100% |███████████████████| 49/49 [746.3ms elapsed, 0s remaining, 65.7 samples/s]      
